In [2]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_instruments_chat,
    get_chords_chat,
    create_song
)
from utils.spotify_manager import (
    get_similar_songs,
    get_track_id_spotify,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    clean_metadata,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [3]:
pc = initialize_pinecone()

In [4]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [5]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [6]:
input_song, input_artist = get_user_input()

In [7]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [8]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [9]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [10]:
input_data

,id,name,artists
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh


Similar songs

In [11]:
tracks_data = get_similar_songs(input_track_id, 5)

In [12]:
for track in tracks_data:
    print(track['name'])


DPM (De Pxta Madre)
Ángel
El Eco de Tu Voz
Te Esperaba
No Se Va


In [13]:
tracks_data = pd.DataFrame(tracks_data)

In [14]:
tracks_data.head(2)

,id,name,artists
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [15]:
filter_existing_vector_id(index, tracks_data)            

In [16]:

tracks_data.reset_index(drop=True, inplace=True)

In [17]:
tracks_data

,id,name,artists
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat


In [18]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [19]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [20]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [21]:
add_info_list_df(input_data, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]"


In [22]:
add_info_list_df(input_data, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]"


Related tracks:

In [23]:
add_info_list_df(df, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[C, G, Am, F, F, C, G, Am]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[A, E, F#m, D, A, E, D, A]"
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,"[C, G, Am, F, C, G, F, C]"


In [24]:
add_info_list_df(df, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[C, G, Am, F, F, C, G, Am]","[G, D, Em, C, G, C, D, Em]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[A, E, F#m, D, A, E, D, A]","[C, G, Am, F, C, G, F, G]"
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, Am, F]"


# API de Spotify para nuevas columnas

Input:

In [25]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008


In [26]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056


In [27]:
insert_lyrics_db(input_data)

(                       id   name               artists  \
 0  06p2Ij0lXD45LODQDCov0u  Rosas  La Oreja de Van Gogh   
 
                        chords                instruments  speechiness  \
 0  [C, G, Am, F, F, G, Em, C]  [C, G, Am, F, C, G, F, G]        0.027   
 
    instrumentalness  key  mode    tempo  loudness  \
 0          0.000008    9     1  106.991    -5.056   
 
                                                theme  
 0  En un día de estos en que suelo pensar\n"hoy v...  ,
 'En un día de estos en que suelo pensar\n"hoy va a ser el día menos pensado",\nnos hemos cruzado, has decidido mirar,\na los ojitos azules que ahora van a tu lado\nDesde el momento en el que te conocí\n\nresumiendo con prisas Tiempo de Silencio\n')

In [28]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [29]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v..."


Related tracks:

In [30]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[C, G, Am, F, F, C, G, Am]","[G, D, Em, C, G, C, D, Em]",0.2610,0.000000
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0456,0.000013
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[A, E, F#m, D, A, E, D, A]","[C, G, Am, F, C, G, F, G]",0.0268,0.000000
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, Am, F]",0.0453,0.000000


In [31]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[C, G, Am, F, F, C, G, Am]","[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[A, E, F#m, D, A, E, D, A]","[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,"[C, G, Am, F, C, G, F, C]","[C, G, Am, F, C, G, Am, F]",0.0453,0.000000,0,1,92.044,-5.370


In [32]:
insert_lyrics_db(df)

(                       id                 name        artists  \
 0  4wJXlFV0oh88ktxE54uqKE  DPM (De Pxta Madre)    Kany García   
 1  2rYjkYQWpn66nf5Ihk7SHJ                Ángel       Elefante   
 2  22t10FTx8y3pC5fjLuKJ8K          Te Esperaba  Carlos Rivera   
 3  4khWEpnaijN0G1x019Uzdr             No Se Va          Morat   
 
                        chords                 instruments  speechiness  \
 0  [C, G, Am, F, F, C, G, Am]  [G, D, Em, C, G, C, D, Em]       0.2610   
 1   [C, G, Am, F, C, G, F, C]   [C, G, Am, F, C, G, F, C]       0.0456   
 2  [A, E, F#m, D, A, E, D, A]   [C, G, Am, F, C, G, F, G]       0.0268   
 3   [C, G, Am, F, C, G, F, C]  [C, G, Am, F, C, G, Am, F]       0.0453   
 
    instrumentalness  key  mode    tempo  loudness  \
 0          0.000000    2     1  169.867    -6.354   
 1          0.000013    1     0  137.003    -4.610   
 2          0.000000    6     1   83.922    -6.722   
 3          0.000000    0     1   92.044    -5.370   
 
                   

In [33]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

In [34]:
chord_map = {
    'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 'E': 4, 
    'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 'Ab': 8, 'A': 9, 
    'A#': 10, 'Bb': 10, 'B': 11
}

Input:

In [35]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v..."


In [36]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [37]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v...","[C, G, Am, F]","[F, G, Em, C]"


In [38]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [39]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v...","[C, G, Am, F]","[F, G, Em, C]"


In [40]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [41]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v...","[C, G, Am, F]","[F, G, Em, C]","[0, 7, 21, 5]","[5, 7, 16, 0]"


In [42]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v...","[C, G, Am, F]","[F, G, Em, C]","[0, 7, 21, 5]","[5, 7, 16, 0]","[-9, -2, 12, -4]","[-4, -2, 7, -9]"


In [43]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar\n""hoy v...","[C, G, Am, F]","[F, G, Em, C]","[0, 7, 21, 5]","[5, 7, 16, 0]","[-9, -2, 12, -4]","[-4, -2, 7, -9]","[7, 14, -16]","[2, 9, -16]"


In [44]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [45]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar ""hoy va...","[C, G, Am, F]","[F, G, Em, C]","[0, 7, 21, 5]","[5, 7, 16, 0]","[-9, -2, 12, -4]","[-4, -2, 7, -9]","[7, 14, -16]","[2, 9, -16]"


Related tracks:

In [46]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [47]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando\nA que el café en la m...,"[C, G, Am, F]","[F, C, G, Am]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada\ncuando menos lo espe...,"[C, G, Am, F]","[C, G, F, C]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722,Te pedí\nCon mi fuerza al universo\nTe escribí...,"[A, E, F#m, D]","[A, E, D, A]"


In [48]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [49]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando\nA que el café en la m...,"[C, G, Am, F]","[F, C, G, Am]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada\ncuando menos lo espe...,"[C, G, Am, F]","[C, G, F, C]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722,Te pedí\nCon mi fuerza al universo\nTe escribí...,"[A, E, F#m, D]","[A, E, D, A]"


In [50]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [51]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando\nA que el café en la m...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada\ncuando menos lo espe...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722,Te pedí\nCon mi fuerza al universo\nTe escribí...,"[A, E, F#m, D]","[A, E, D, A]","[9, 4, 18, 2]","[9, 4, 2, 9]"


In [52]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando\nA que el café en la m...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]","[-2, 5, 19, 3]","[3, -2, 5, 19]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada\ncuando menos lo espe...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-1, 6, 20, 4]","[-1, 6, 4, -1]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722,Te pedí\nCon mi fuerza al universo\nTe escribí...,"[A, E, F#m, D]","[A, E, D, A]","[9, 4, 18, 2]","[9, 4, 2, 9]","[3, -2, 12, -4]","[3, -2, -4, 3]"
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,"[C, G, Am, F, C, G, Am, F]",0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme\nY tan difícil olvida...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]"


In [53]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando\nA que el café en la m...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]","[-2, 5, 19, 3]","[3, -2, 5, 19]","[7, 14, -16]","[-5, 7, 14]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada\ncuando menos lo espe...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-1, 6, 20, 4]","[-1, 6, 4, -1]","[7, 14, -16]","[7, -2, -5]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722,Te pedí\nCon mi fuerza al universo\nTe escribí...,"[A, E, F#m, D]","[A, E, D, A]","[9, 4, 18, 2]","[9, 4, 2, 9]","[3, -2, 12, -4]","[3, -2, -4, 3]","[-5, 14, -16]","[-5, -2, 7]"
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,"[C, G, Am, F, C, G, Am, F]",0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme\nY tan difícil olvida...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[0, 7, 21, 5]","[0, 7, 5, 0]","[7, 14, -16]","[7, -2, -5]"


In [54]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [55]:
df_vectorial = df_vectorial.fillna(0)

In [56]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,"[G, D, Em, C, G, C, D, Em]",0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]","[-2, 5, 19, 3]","[3, -2, 5, 19]","[7, 14, -16]","[-5, 7, 14]"
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,"[C, G, Am, F, C, G, F, C]",0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,"[C, G, Am, F]","[C, G, F, C]","[0, 7, 21, 5]","[0, 7, 5, 0]","[-1, 6, 20, 4]","[-1, 6, 4, -1]","[7, 14, -16]","[7, -2, -5]"
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,"[C, G, Am, F, C, G, F, G]",0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,"[A, E, F#m, D]","[A, E, D, A]","[9, 4, 18, 2]","[9, 4, 2, 9]","[3, -2, 12, -4]","[3, -2, -4, 3]","[-5, 14, -16]","[-5, -2, 7]"


In [57]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]","[C, G, Am, F, C, G, F, G]",0.027,0.000008,9,1,106.991,-5.056,"En un día de estos en que suelo pensar ""hoy va...","[C, G, Am, F]","[F, G, Em, C]","[0, 7, 21, 5]","[5, 7, 16, 0]","[-9, -2, 12, -4]","[-4, -2, 7, -9]","[7, 14, -16]","[2, 9, -16]"


# Backup csv

In [58]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song_starter/utils/functions.py:289: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song_starter/utils/functions.py:295: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [59]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [60]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [61]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,06p2Ij0lXD45LODQDCov0u,Rosas,La Oreja de Van Gogh,"[C, G, Am, F, F, G, Em, C]",0.027,0.000008,9,1,106.991,-5.056,...,-4,-2,7,-9,7,14,-16,2,9,-16


In [62]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [63]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [64]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,...,3,-2,5,19,7,14,-16,-5,7,14
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,...,-1,6,4,-1,7,14,-16,7,-2,-5
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,...,3,-2,-4,3,-5,14,-16,-5,-2,7
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme Y tan difícil olvidar...,...,0,7,5,0,7,14,-16,7,-2,-5


In [65]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [66]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,...,3,-2,5,19,7,14,-16,-5,7,14
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,...,-1,6,4,-1,7,14,-16,7,-2,-5
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,...,3,-2,-4,3,-5,14,-16,-5,-2,7
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme Y tan difícil olvidar...,...,0,7,5,0,7,14,-16,7,-2,-5


In [67]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,...,3,-2,5,19,7,14,-16,-5,7,14
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,...,-1,6,4,-1,7,14,-16,7,-2,-5
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,...,3,-2,-4,3,-5,14,-16,-5,-2,7


# Create column for the text to create the embeddings

Input:

In [68]:
input_expanded=to_embbed_text_column(input_expanded)

In [69]:
input_expanded['to_embbed_text']

0    name: Rosas artists: La Oreja de Van Gogh chor...
Name: to_embbed_text, dtype: object

Related songs:

In [70]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [71]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,...,-2,5,19,7,14,-16,-5,7,14,name: DPM (De Pxta Madre) artists: Kany García...
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,...,6,4,-1,7,14,-16,7,-2,-5,name: Ángel artists: Elefante speechiness: 0.0...
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,...,-2,-4,3,-5,14,-16,-5,-2,7,name: Te Esperaba artists: Carlos Rivera speec...
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme Y tan difícil olvidar...,...,7,5,0,7,14,-16,7,-2,-5,name: No Se Va artists: Morat speechiness: 0.0...


# Text for embeddings to chat gpt

In [72]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [73]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [74]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [75]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [76]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,...,-2,5,19,7,14,-16,-5,7,14,name: DPM (De Pxta Madre) artists: Kany García...
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,...,6,4,-1,7,14,-16,7,-2,-5,name: Ángel artists: Elefante speechiness: 0.0...
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,...,-2,-4,3,-5,14,-16,-5,-2,7,name: Te Esperaba artists: Carlos Rivera speec...
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme Y tan difícil olvidar...,...,7,5,0,7,14,-16,7,-2,-5,name: No Se Va artists: Morat speechiness: 0.0...


Related songs:

In [77]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [78]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [79]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [80]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/4 [00:00<?, ?it/s]

In [81]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,4wJXlFV0oh88ktxE54uqKE,DPM (De Pxta Madre),Kany García,0.2610,0.000000,2,1,169.867,-6.354,Me fuiste acostumbrando A que el café en la ma...,...,5,19,7,14,-16,-5,7,14,"[0.019171737134456635, 0.026316126808524132, 0...","{'name': 'DPM (De Pxta Madre)', 'artists': 'Ka..."
1,2rYjkYQWpn66nf5Ihk7SHJ,Ángel,Elefante,0.0456,0.000013,1,0,137.003,-4.610,Te encontre de madrugada cuando menos lo esper...,...,4,-1,7,14,-16,7,-2,-5,"[0.010688980109989643, -0.008711980655789375, ...","{'name': 'Ángel', 'artists': 'Elefante', 'spee..."
2,22t10FTx8y3pC5fjLuKJ8K,Te Esperaba,Carlos Rivera,0.0268,0.000000,6,1,83.922,-6.722,Te pedí Con mi fuerza al universo Te escribí e...,...,-4,3,-5,14,-16,-5,-2,7,"[0.020393310114741325, 0.0019659537356346846, ...","{'name': 'Te Esperaba', 'artists': 'Carlos Riv..."
3,4khWEpnaijN0G1x019Uzdr,No Se Va,Morat,0.0453,0.000000,0,1,92.044,-5.370,Tan fácil que enamorarme Y tan difícil olvidar...,...,5,0,7,14,-16,7,-2,-5,"[0.01817324198782444, 0.04055362194776535, -0....","{'name': 'No Se Va', 'artists': 'Morat', 'spee..."


In [82]:
df_para_pinecone

,id,values,metadata
0,4wJXlFV0oh88ktxE54uqKE,"[0.019171737134456635, 0.026316126808524132, 0...","{'name': 'DPM (De Pxta Madre)', 'artists': 'Ka..."
1,2rYjkYQWpn66nf5Ihk7SHJ,"[0.010688980109989643, -0.008711980655789375, ...","{'name': 'Ángel', 'artists': 'Elefante', 'spee..."
2,22t10FTx8y3pC5fjLuKJ8K,"[0.020393310114741325, 0.0019659537356346846, ...","{'name': 'Te Esperaba', 'artists': 'Carlos Riv..."
3,4khWEpnaijN0G1x019Uzdr,"[0.01817324198782444, 0.04055362194776535, -0....","{'name': 'No Se Va', 'artists': 'Morat', 'spee..."


# Query search in pinecone

In [83]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [84]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [85]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: guitar saxophone, Tempo: 131.685, Speechiness: 0.0469, Loudness: -4.09',
 'Intruments: guitar drums, Tempo: 117.252, Speechiness: 0.07, Loudness: -7.16',
 'Intruments: acoustic guitar keyboard, Tempo: 149.72, Speechiness: 0.0489, Loudness: -5.518']

In [86]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.205643758
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.200729102
Letra: The song "El Encuentro" by Biela explores themes of connection, longing, and intimacy. It captures the emotions surrounding a significant meeting between two individuals, emphasizing the beauty and complexity of human relationships. Through its lyrical imagery, it conveys a sense of urgency and the profound impact such encounters can have on one’s life. Overall, the song reflects on the joys and challenges of love and the deep bonds formed through shared experiences., 
 Score: 0.190816268
Letra: I'm sor— I'm sorry that you've come to me In dreams like it's the only way That you can find to talk to me, 
 Score: 0.189126477
Letra: I already died I'm singing from the other side Underneath love Avoidance and pride, 
 Score: 

In [87]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [88]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [89]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, F Am C G, 
 Score: 0.467369378
Chords: C G Am F, C F G Am, 
 Score: 0.458404183
Chords: C G Am F, C G F F, 
 Score: 0.456994683
Chords: C G Am F, D A Bm G, 
 Score: 0.450550675
Chords: 2023 Remaster C G Am, F C G F, 
 Score: 0.44867456


In [90]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [91]:
chord_wheels

['Chord wheel: C G Am F, F Am C G',
 'Chord wheel: C G Am F, C F G Am',
 'Chord wheel: C G Am F, C G F F',
 'Chord wheel: C G Am F, D A Bm G',
 'Chord wheel: 2023 Remaster C G Am, F C G F']

In [92]:
input_data['theme'][0]

'En un día de estos en que suelo pensar "hoy va a ser el día menos pensado", nos hemos cruzado, has decidido mirar, a los ojitos azules que ahora van a tu lado Desde el momento en el que te conocí  resumiendo con prisas Tiempo de Silencio '

In [93]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [94]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")



for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

lyrics_list.append(input_data['theme'][0])

Lyrics: En un día de estos en que suelo pensar "hoy va a ser el día menos pensado", nos hemos cruzado, has decidido mirar, a los ojitos azules que ahora van a tu lado Desde el momento en el que te conocí  resumiendo con prisas Tiempo de Silencio  06p2Ij0lXD45LODQDCov0u Score: 0.51974
Lyrics: De vuelta a la rutina Nos vestimos sin hablarnos La prisa puede más que la pasión Me pesa cada día Los recuerdos y los ratos Como dardos clavan este corazón 5Z8lZ3FhHDMF85kZXXGhFZ Score: 0.481440663
Lyrics: Dime que nunca lo has pensado dime que nunca has tonteado con la posibilidad de dar de arder el cuadro que te han colgado en la pared Vente conmigo al otro lado  cruza la linea de mi mano  39eeQfGBXwK0qKQvwGHjoI Score: 0.475335836
Lyrics: The theme of "Si No Te Hubieras Ido" revolves around the profound longing and heartbreak experienced after a loved one departs, reflecting on the what-ifs and missed moments that haunt the narrator. The song captures the deep emotional impact of loss and the de

In [95]:
lyrics_list

['Lyrics: En un día de estos en que suelo pensar "hoy va a ser el día menos pensado", nos hemos cruzado, has decidido mirar, a los ojitos azules que ahora van a tu lado Desde el momento en el que te conocí  resumiendo con prisas Tiempo de Silencio ',
 'Lyrics: De vuelta a la rutina Nos vestimos sin hablarnos La prisa puede más que la pasión Me pesa cada día Los recuerdos y los ratos Como dardos clavan este corazón',
 'Lyrics: Dime que nunca lo has pensado dime que nunca has tonteado con la posibilidad de dar de arder el cuadro que te han colgado en la pared Vente conmigo al otro lado  cruza la linea de mi mano ',
 'Lyrics: The theme of "Si No Te Hubieras Ido" revolves around the profound longing and heartbreak experienced after a loved one departs, reflecting on the what-ifs and missed moments that haunt the narrator. The song captures the deep emotional impact of loss and the desire for a second chance, emphasizing the enduring connection despite physical absence.',
 'Lyrics: Todo el 

In [96]:
nueva_cancion = create_song(lyrics_list, chord_wheels, details)
print(nueva_cancion)

### Título de la canción: "El Día Menos Pensado"

### Tempo: 131 BPM

### Instrumentos principales: Guitarra y bajo

### Potencia: Media

---

#### Letra de la canción y sus acordes:

**Verso 1**  
(C) En un día cualquiera, (G) nos volvimos a ver,  
(Am) Tus ojos me encontraron, (F) sin quererlo entender.  
(C) La rutina nos pesa, (G) el silencio es cruel,  
(Am) Pero al cruzar miradas, (F) todo vuelve a nacer.  

**Coro**  
(C) Hoy es el día (G) menos esperado,  
(Am) donde el tiempo (F) se ha detenido,  
(C) y en tus ojos (G) yo me he encontrado,  
(Am) un segundo (F) que nunca he perdido.  

**Verso 2**  
(C) Entre recuerdos y dudas, (G) el corazón se aferra,  
(Am) a los momentos fugaces, (F) a la luz que destella.  
(C) Dime si alguna vez, (G) pensaste en volver,  
(Am) a cruzar esa línea, (F) y juntos renacer.  

**Coro final**  
(C) Hoy es el día (G) menos esperado,  
(Am) donde el tiempo (F) se ha detenido,  
(C) y en tus ojos (G) yo me he encontrado,  
(Am) un segundo (F) que 

In [97]:
nueva_cancion.content

AttributeError: 'str' object has no attribute 'content'

In [99]:
response.choices[0].message.content

'### Título de la canción: "Entre Amapolas y Lunas"\n\n**Tempo**: 131 BPM\n\n**Instrumentos principales**: Guitarra y saxofón\n\n**Potencia**: Media\n\n---\n\n**Verso 1**  \n(C)  \nYa está ahí la Luna,  \n(G)  \niluminando la soledad,  \n(Am)  \nme pierdo en tus colores,  \n(F)  \nbuscando un hogar.\n\n**Coro**  \n(F)  \nEntre amapolas y lunas,  \n(Am)  \nel valor que me falta,  \n(C)  \nen tus manos encuentro,  \n(G)  \nla vida que me abraza.\n\n**Verso 2**  \n(C)  \nTriste como un perro,  \n(G)  \nen la autopista del adiós,  \n(Am)  \npero tus ojos rojos,  \n(F)  \nme guían sin voz.\n\n**Coro final**  \n(F)  \nEntre amapolas y lunas,  \n(Am)  \nel valor que me falta,  \n(C)  \nen tus manos encuentro,  \n(G)  \nla vida que me abraza.\n\n--- \n\nEsta canción refleja la búsqueda de conexión y el anhelo de encontrar compañía en medio de la soledad, con un ritmo medio que acompaña la intensidad emocional de la letra.'